In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [7]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

"Here's a curious fact about Cristiano Ronaldo, focusing on his dedication to physical perfection:\n\nRonaldo reportedly has the body fat percentage of a supermodel (around 7-10%, often cited as even lower than that). What's *curious* about it, beyond the sheer discipline it requires, is that he achieves this while reportedly having five or six small meals a day, rather than the restrictive diets many might assume. It demonstrates an incredible understanding of his own metabolism and the precise fueling his body needs for peak performance, almost like a finely tuned machine. He avoids sugary drinks and processed foods, of course, but the *frequency* of his eating, combined with the body fat percentage, is unusual and highlights his unique approach. It highlights that it is not just *what* he eats, but *how* and *when* that are key."

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [8]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

"Here's a curious fact about Cristiano "

# Combining LCEL Chains

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

chatModel = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()

In [11]:
chain.invoke("pawan Kalyan")

'Pawan Kalyan is a prominent Indian actor, politician, and founder of the Jana Sena Party.'

# Combined chain

In [17]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [19]:
composed_chain.invoke({"politician": "pawan Kalyan"})

'This question is a bit of a play on words, and it highlights the ambiguity of the phrase "positive for humanity." Here\'s a breakdown of how to interpret it, and why there\'s no single correct answer:\n\n*   **"Positive for" in a medical sense:** This usually refers to testing positive for a disease or condition. Obviously, "humanity" isn\'t a disease one can test positive for. Pawan Kalyan is human, so in that literal, biological sense, he is "positive for humanity".\n*   **"Positive for humanity" in a figurative sense:** This interpretation asks if Pawan Kalyan\'s actions, career, and political stances have been a net positive influence on humankind (or at least, on the people he interacts with/represents). This is entirely subjective and open to debate.\n\n    *   **Arguments that he *is* "positive for humanity":**\n        *   **Philanthropy:** He has been involved in charitable activities and disaster relief efforts. His fans and supporters often point to his generosity.\n       

## Another example: a chain inside another chain

In [21]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)



chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "pawan Kalyan", "language": "French"})

'Inde est en Asie.'

# LCEL chain at work in a typical RAG app

In [22]:
from langchain_google_genai import GoogleGenerativeAI
llm1 = GoogleGenerativeAI(model="gemini-2.0-flash-lite-preview-02-05")

In [23]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm1
    | StrOutputParser()
)

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\Swapna\anaconda3\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [24]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition involves breaking down complex tasks into smaller, simpler steps. This is often achieved by instructing a model to "think step by step" to manage the task. It can be done by LLM with simple prompting, using task-specific instructions, or with human input.\n'